In [18]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from torchvision.transforms import Normalize
from torchvision.transforms import ToTensor

In [19]:
# Load CNN and UNET
from CNN_Mask_to_Tens import CNN
from Unet_nn import UNet

ImportError: cannot import name 'UNet' from 'Unet_nn' (d:\One drive\OneDrive - TU Eindhoven\TUE 2\4AI00\4AI00\Blender_Automation\Unet_nn.py)

In [5]:
# Load and preprocess the input image
input_image = Image.open('Test_Images\Train_10200\drop_s30_v5_r0.5_str6_pos0_cam1.png')  # Replace 'input_image.jpg' with the path to your input image file

In [6]:
# UNET PART


# OUPUT UNET SHOULD BE CONVERTED USING THE FOLLOWING TRANSFORM

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.4753, 0.4753, 0.4753])
])

input_image = input_image.convert('RGB')
input_tensor = transform(input_image)

In [7]:
# Load model
model = CNN()

# Load the trained weights
model.load_state_dict(torch.load('trained_model_Train_510.pt')) # Change trained_model to use desired weights
# Set the model in evaluation mode

model.eval()


input_image = input_image.convert('RGB')
input_tensor = transform(input_image)

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)

# Convert the output to a readable format
predicted_value = output.item()

# Print the predicted value
print('Predicted value:', predicted_value)

Predicted value: 61.35445785522461
